In [1]:
# widen jupyter notebook window
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

# check environment
import os
print(f'Conda Environment: ' + os.environ['CONDA_DEFAULT_ENV'])

Conda Environment: fr_env


In [2]:
import matplotlib.pyplot as plt

import numpy as np
import torch

import scipy.io
import scipy.interpolate

import sklearn

from tqdm.notebook import tqdm

import copy
import time
import gc

In [3]:
import sys
sys.path.append('/n/data1/hms/neurobio/sabatini/rich/github_repos/')

%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import *

%load_ext autoreload
%autoreload 2
from NBAP import import_data, align_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import cascade2p
from cascade2p import checks
checks.check_packages()
from cascade2p import cascade # local folder
from cascade2p.utils import plot_dFF_traces, plot_noise_level_distribution, plot_noise_matched_ground_truth

	YAML reader installed (version 0.17.19).


Using TensorFlow backend.
2021-12-29 03:42:38.934846: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rh183/miniconda3/envs/fr_env/lib/python3.8/site-packages/cv2/../../lib64:
2021-12-29 03:42:38.934873: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


	Keras installed (version 2.3.1).
	Tensorflow installed (version 2.3.0).


In [5]:
import torchinterp1d

In [6]:
from pathlib import Path

dir_S2p = Path(r'/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210409/baseline/suite2p/plane0/')

In [1]:
dir_batch = ['/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210409/baseline/suite2p/plane0/',
             '/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210410/exp/suite2p/plane0/',
             '/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210411/exp/suite2p/plane0/',
             '/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210412/exp/suite2p/plane0/',
             '/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210413/exp/suite2p/plane0/',
             '/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210414/exp/suite2p/plane0/',
             '/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210415/exp/suite2p/plane0/',
             '/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210416/exp/suite2p/plane0/',
             '/n/data1/hms/neurobio/sabatini/rich/data/res2p/round_5_experiments/mouse_2_6/scanimage/20210417/exp/suite2p/plane0/',
            ]

In [ ]:
spike_prob_all = []
for iter_day,dir_S2p in enumerate(dir_batch):

    F , Fneu , iscell , ops , spks , stat , num_frames_S2p = import_data.import_S2p(dir_S2p)

    channelOffset_correction = 150
    percentile_baseline = 10

    dFoF , dF , F_neuSub , F_baseline = ca2p_preprocessing.make_dFoF(   F=F + channelOffset_correction,
                                                                        Fneu=Fneu + channelOffset_correction,
                                                                        neuropil_fraction=0.7,
                                                                        percentile_baseline=percentile_baseline,
                                                                        multicore_pref=True,
                                                                        verbose=True)

    dFoF_smooth = timeSeries.convolve_along_axis(dFoF,
                                                kernel=math_functions.gaussian(np.arange(-15,15), 0, sig=50/30, plot_pref=False)[0],
                                                axis=1,
                                                mode='same',
                                                multicore_pref=True,
                                                verbose=True)

    spike_prob = np.concatenate([cascade.predict(model_name='Global_EXC_30Hz_smoothing50ms_causalkernel',
                                                 traces=batch, 
                                                 model_folder='/n/data1/hms/neurobio/sabatini/rich/github_repos/Cascade/Pretrained_models', 
                                                 padding=0
                                                ) for batch in indexing.make_batches(dFoF_smooth, num_batches=20)], axis=0)

    np.save(f'/n/data1/hms/neurobio/sabatini/rich/spike_prob_{iter_day}.npy', np.single(spike_prob))
    spike_prob_all.append(np.single(spike_prob))

In [9]:
np.save(f'/n/data1/hms/neurobio/sabatini/rich/spike_prob_all.npy', spike_prob_all)